In [0]:
races_df = spark.read.table('formula1_project.silver.races_transformed')
races_df = races_df.filter(races_df.race_year ==2019)
races_df.display()

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                   StructField("circuitRef", StringType(), True),
                                   StructField("name", StringType(), True),
                                   StructField("location", StringType(), True),
                                   StructField("country", StringType(), True),
                                   StructField("lat", DoubleType(), True),
                                   StructField("lng", DoubleType(), True),
                                   StructField("alt", IntegerType(), True),
                                   StructField("url", StringType(), True)])

circuits_df = spark.read.option("header", "true") \
.schema(circuits_schema) \
.csv("abfss://bronze@mjuanworkshopetl.dfs.core.windows.net/circuits.csv") 

In [0]:
circuits_df = circuits_df.withColumnRenamed('circuitId', 'circuit_id') \
.withColumnRenamed('circuitRef', 'circuit_ref') \
.withColumnRenamed('latitude', 'latitude') \
.withColumnRenamed('lng', 'longitude') \
.withColumnRenamed('alt', 'altitude') \
.filter(circuits_df.circuit_id < 70)
circuits_df.display()

In [0]:
race_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, 'inner'). \
select(circuits_df.name, circuits_df.location, circuits_df.country, races_df.name, races_df.round )
race_circuits_df.display()

In [0]:
race_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, 'left'). \
filter(races_df.circuit_id.isNull()) .\
select(circuits_df.name, circuits_df.location, circuits_df.country, races_df.name, races_df.round )
race_circuits_df.display()

In [0]:
race_circuits_df = races_df.alias("a").join(circuits_df.alias("b"), races_df.circuit_id == circuits_df.circuit_id, 'left'). \
filter("b.circuit_id is NULL") .\
select(circuits_df.name, circuits_df.location, circuits_df.country, races_df.name, races_df.round )
race_circuits_df.display()